# Scheduling Model
### Conjuntos
- $A$ es el conjunto de tareas
- $O$ es el conjunto de órdenes
- $M$ es el conjunto de máquinas

### Parámetros
- $d_i$ duración de la tarea $i\in A$
- $\tau_i$ tiempo más temprano para comenzar la tarea $i\in A$
- $T_o$ tiempo de entrega de la orden $o\in O$
- $p(i)$ conjunto de tareas que preceden a $i\in A$
- $m(i)$ conjunto de máquinas en las que se puede realizar la tarea $i\in A$
- $r(k) = \{i\in A \ | \ k \in m(i) \}$ conjunto de tareas que se pueden realizar en la máquina $k\in M$
- $r_0(k) = r(k) \cup \{0\}$
- $w(o)$ conjunto de tareas que componen la orden $o\in O$

### Variables
- $x_{ijk}$ variable binaria igual 1 cuando en la máquina $k\in M$ la tarea $j\in r_0(k)$ se programa despues de la tarea $i\in r_0(k):i\neq j$
- $t_{ik}$ tiempo en el que se comienza a trabajar la tarea $i\in A$ en la máquina $k\in m(i)$

### Modelo
\begin{align}
\min \quad & \max_{i\in A, \ k \in m(i)} \{ t_{ik}+d_i \} \\
\text{s.t.} \quad & \sum_{k\in m(j)} \sum_{i\in r_0(k)} x_{ijk} = 1 && j \in A \\
& \sum_{k\in m(i)} \sum_{j\in r_0(k)} x_{ijk} = 1 && i \in A \\
& \sum_{i\in r_0(k)} x_{ihk} = \sum_{j\in r_0(k)} x_{hjk} && h\in A,\ k\in m(h) \\
& \sum_{j\in r(k)} x_{0jk} \leq 1 && k \in M \\
& \sum_{i\in r(k)} x_{i0k} \leq 1 && k \in M \\
& x_{ijk} = 1 \ \Rightarrow \ t_{ik} + d_i \leq t_{jk} && k\in M,\ i\in r(k),\ j\in r(k),\ i\neq j\\
& t_{ik} \geq \tau_i && i\in A,\ k \in m(i) \\
& t_{ig} +d_i \leq t_{jh} && j \in A , \ i \in p(j) , \ g \in m(i), \ h \in m(j) \\
& x_{ijk} \in\{0,1\} && k\in M,\ i\in r_0(k),\ j\in r_0(k),\ i\neq j
\end{align}

In [1]:
import collections
from docplex.mp.model import Model

In [2]:
Order = collections.namedtuple('Order', ['tasks', 'due_date'])
Task = collections.namedtuple('Task',['machines','duration',
                                      'earliest_start','precedence'])

In [3]:
tasks = {1: Task([1, 2], 5, 3, []),
         2: Task([2, 3], 7, 3, [1]),
         3: Task([1, 4], 8, 3, [1]),
         4: Task([5], 2, 3, []),
         5: Task([1, 5, 3], 5, 0, [4]),
         6: Task([5], 9, 10, []),
         7: Task([5], 4, 10, [2, 3]),
         8: Task([5], 4, 10, []),
         9: Task([3, 4], 4, 10, [8]),
         10:Task([5], 4, 10, [9])}

In [4]:
orders = {1: Order(tasks=[1, 2, 3, 7], due_date=5),
          2: Order([4, 5], 10),
          3: Order([6], 6),
          4: Order([8, 9, 10], 5)}

In [5]:
machines = [1, 2, 3, 4, 5]

In [6]:
tasks_per_machine = {}
for k in machines:
    tasks_this_machine = []
    for i, task in tasks.items():
        if k in task.machines:
            tasks_this_machine.append(i)
    tasks_per_machine[k] = tasks_this_machine
tasks_per_machine

{1: [1, 3, 5], 2: [1, 2], 3: [2, 5, 9], 4: [3, 9], 5: [4, 5, 6, 7, 8, 10]}

In [7]:
tuples = [(i,j,k) for k in machines 
                  for i in tasks_per_machine[k]+[0] 
                  for j in tasks_per_machine[k]+[0]
                  if i!=j]

In [8]:
mdl = Model('Scheduling')

In [9]:
x = mdl.binary_var_dict(tuples, name='x')
t = mdl.continuous_var_dict([(i,k) for i, task in tasks.items() 
                                   for k in task.machines],name='t')
y = mdl.continuous_var(name='y')

In [10]:
mdl.minimize(y)

$$\sum_{k\in m(j)} \sum_{i\in r_0(k)} x_{ijk} = 1 \qquad j \in A$$

In [11]:
for j, task_j in tasks.items():
    mdl.add_constraint(mdl.sum(x[(i,j,k)] for k in task_j.machines 
                                          for i in tasks_per_machine[k]+[0]
                                          if i!=j)==1,ctname='in_%d'%j)

$$\sum_{k\in m(i)} \sum_{j\in r_0(k)} x_{ijk} = 1 \qquad i \in A$$

In [12]:
for i, task_i in tasks.items():
    mdl.add_constraint(mdl.sum(x[(i,j,k)] for k in task_i.machines
                                          for j in tasks_per_machine[k]+[0]
                                          if j!=i)==1,ctname='out_%d'%i)

$$\sum_{i\in r_0(k)} x_{ihk} = \sum_{j\in r_0(k)} x_{hjk} \qquad h\in A,\ k\in m(h)$$

In [13]:
for h, task_h in tasks.items():
    for k in task_h.machines:
        mdl.add_constraint(
            mdl.sum(x[(i,h,k)] for i in tasks_per_machine[k]+[0] if i!=h)==
            mdl.sum(x[(h,j,k)] for j in tasks_per_machine[k]+[0] if j!=h),
            ctname='flow_%d_%d'%(h,k)
        )

$$\sum_{j\in r(k)} x_{0jk} \leq 1 \qquad  k \in M$$
$$\sum_{i\in r(k)} x_{i0k} \leq 1 \qquad k \in M $$

In [14]:
for k in machines:
    mdl.add_constraint(mdl.sum(x[(0,j,k)] for j in tasks_per_machine[k])<=1,
                       ctname='start_%d'%k)
    mdl.add_constraint(mdl.sum(x[(i,0,k)] for i in tasks_per_machine[k])<=1,
                       ctname='end_%d'%k)

$$ x_{ijk} = 1 \ \Rightarrow \ t_{ik} + d_i \leq t_{jk} \qquad  k\in M,\ i\in r(k),\ j\in r(k),\ i\neq j$$

In [15]:
for i,j,k in tuples:
    if i!=0 and j!=0:
        mdl.add_indicator(x[(i,j,k)],
                          t[(i,k)] + tasks[i].duration <= t[(j,k)],
                          name='order_%d_%d_%d'%(i,j,k))

$$ t_{ig} +d_i \leq t_{jh} \qquad j \in A , \ i \in p(j) , \ g \in m(i), \ h \in m(j) $$

In [16]:
for j, task_j in tasks.items():
    for i in task_j.precedence:
        task_i = tasks[i]
        for g in task_i.machines:
            for h in task_j.machines:
                mdl.add_constraint(t[(i,g)] + tasks[i].duration <= t[(j,h)],
                                   ctname='pre_%d<%d_%d.%d'%(i,j,g,h))

$$ t_{ik} \geq \tau_i \qquad i\in A,\ k \in m(i) $$
$$ t_{ik} +d_i \leq y \qquad i\in A,\ k \in m(i) $$

In [17]:
for i, task_i in tasks.items():
    for k in task_i.machines:
        mdl.add_constraint(t[(i,k)] >= task_i.earliest_start,
                           ctname='earliest_start%d_%d'%(i,k))
        mdl.add_constraint(t[(i,k)] + task_i.duration <= y,
                           ctname='makespan_%d_%d'%(i,k))

In [18]:
solucion = mdl.solve(log_output=True)
mdl.get_solve_status()

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Tried aggregator 2 times.
MIP Presolve eliminated 28 rows and 10 columns.
MIP Presolve modified 80 coefficients.
Aggregator did 23 substitutions.
Reduced MIP has 87 rows, 108 columns, and 323 nonzeros.
Reduced MIP has 73 binaries, 0 generals, 0 SOSs, and 40 indicators.
Presolve time = 0.02 sec. (0.52 ticks)
Probing time = 0.00 sec. (0.11 ticks)
Tried aggregator 1 time.
Reduced MIP has 87 rows, 108 columns, and 323 nonzeros.
Reduced MIP has 73 binaries, 0 generals, 0 SOSs, and 40 indicators.
Presolve time = 0.00 sec. (0.22 ticks)
Probing time = 0.00 sec. (0.11 ticks)
Clique table members: 163.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.02 sec. (0.16 ticks)

        Nodes                                  

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [19]:
mdl.objective_value

31.0

In [20]:
solucion.display()

solution for: Scheduling
y: 31.000
x_(3, 0, 1) = 1
x_(10, 0, 5) = 1
x_(0, 4, 5) = 1
x_(0, 3, 1) = 1
x_(1, 0, 2) = 1
t_(1, 1) = 3.000
t_(1, 2) = 3.000
t_(2, 2) = 8.000
t_(2, 3) = 10.000
t_(3, 1) = 15.000
t_(3, 4) = 8.000
x_(2, 0, 3) = 1
x_(5, 2, 3) = 1
t_(5, 5) = 26.000
t_(5, 3) = 5.000
t_(6, 5) = 14.000
t_(7, 5) = 23.000
t_(8, 5) = 10.000
t_(9, 3) = 14.000
x_(0, 5, 3) = 1
t_(10, 5) = 27.000
y = 31.000
x_(9, 0, 4) = 1
x_(0, 9, 4) = 1
x_(0, 1, 2) = 1
x_(4, 8, 5) = 1
t_(9, 4) = 14.000
x_(6, 7, 5) = 1
t_(4, 5) = 3.000
x_(7, 10, 5) = 1
x_(8, 6, 5) = 1
t_(5, 1) = 26.000


In [21]:
from bokeh.palettes import Spectral10
import bokeh.palettes as bp
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.models import HoverTool

In [22]:
output_notebook()
palette = bp.all_palettes['Spectral'][len(orders)]
color = 0
task_color = {}
for o in orders.values():
    for i in o.tasks:
        task_color[i] = palette[color]
    color = color + 1
graph_data = {'left': [], 'bottom': [], 'right': [],
              'top': [], 'task': [], 'color': []}
for k in machines:
    arcos_activos = [(i, j) for i in tasks_per_machine[k]+[0]
                            for j in tasks_per_machine[k]+[0]
                            if i!=j and x[i,j,k].solution_value > 0.9]
    for i, j in arcos_activos:
        if i!=0:
            graph_data['left'].append(t[(i, k)].solution_value)
            graph_data['right'].append(t[(i, k)].solution_value + 
                                       tasks[i].duration)
            graph_data['bottom'].append(k - 0.25)
            graph_data['top'].append(k + 0.25)
            graph_data['task'].append(i)
            graph_data['color'].append(task_color[i])
source = ColumnDataSource(data=graph_data)
hover = HoverTool(tooltips = [("task", "@task"),('start', '@left'),
                              ('end', '@right')])      
p = figure(plot_width=800, plot_height=400, 
           tools=[hover, 'save,xzoom_in,xzoom_out,reset,xpan'])
p.quad(left='left', bottom='bottom',right='right', top='top', 
       color='color', line_color='black', alpha=0.8, source=source)
show(p, notebook_handle=True)

Loading BokehJS ...